In [1]:
pip install pyspark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 9.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=e86abe2c28a7c2bd9def12727dc324c28b9257d1ea93547dacc0299f1a1987c4
  Stored in directory: /root/.cache/pip/wheels/b1/59/a0/a1a0624b5e865fd389919c1a10f53aec9b12195d6747710baf
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.master("local[1]").appName("ml").getOrCreate()

In [4]:
sc = spark.sparkContext

In [5]:
df = spark.read.options(header=True,inferSchema=True).csv("Social_Network_Ads.csv")

In [6]:
df

DataFrame[User ID: int, Gender: string, Age: double, EstimatedSalary: double, Purchased: int]

In [7]:
df.count()

400

In [8]:
df.summary().show()

+-------+-----------------+------+------------------+----------------+------------------+
|summary|          User ID|Gender|               Age| EstimatedSalary|         Purchased|
+-------+-----------------+------+------------------+----------------+------------------+
|  count|              400|   400|               400|             400|               400|
|   mean|  1.56915397575E7|  null|            37.655|         69742.5|            0.3575|
| stddev|71658.32158119006|  null|10.482876597307927|34096.9602824248|0.4798639635968691|
|    min|         15566689|Female|              18.0|         15000.0|                 0|
|    25%|         15625395|  null|              29.0|         43000.0|                 0|
|    50%|         15694288|  null|              37.0|         70000.0|                 0|
|    75%|         15750335|  null|              46.0|         88000.0|                 1|
|    max|         15815236|  Male|              60.0|        150000.0|                 1|
+-------+-

In [9]:
df.describe()

DataFrame[summary: string, User ID: string, Gender: string, Age: string, EstimatedSalary: string, Purchased: string]

In [14]:
df.groupby('Purchased').count().show()

+---------+-----+
|Purchased|count|
+---------+-----+
|        1|  143|
|        0|  257|
+---------+-----+



In [15]:
df.groupby('Gender').count().show()

+------+-----+
|Gender|count|
+------+-----+
|Female|  204|
|  Male|  196|
+------+-----+



In [16]:
pdf = df.toPandas()

In [17]:
pdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   User ID          400 non-null    int32  
 1   Gender           400 non-null    object 
 2   Age              400 non-null    float64
 3   EstimatedSalary  400 non-null    float64
 4   Purchased        400 non-null    int32  
dtypes: float64(2), int32(2), object(1)
memory usage: 12.6+ KB


In [18]:
pdf.isnull().sum()

User ID            0
Gender             0
Age                0
EstimatedSalary    0
Purchased          0
dtype: int64

In [42]:
from pyspark.ml.feature import MinMaxScaler,VectorAssembler

In [25]:
required_features = ['Age','EstimatedSalary']

In [27]:
assembler = VectorAssembler(inputCols = required_features, outputCol = 'features')

In [28]:
transformed_data = assembler.transform(df)

In [43]:
transformed_data.show(5)


+--------+------+----+---------------+---------+--------------+
| User ID|Gender| Age|EstimatedSalary|Purchased|      features|
+--------+------+----+---------------+---------+--------------+
|15624510|  Male|19.0|        19000.0|        0|[19.0,19000.0]|
|15810944|  Male|35.0|        20000.0|        0|[35.0,20000.0]|
|15668575|Female|26.0|        43000.0|        0|[26.0,43000.0]|
|15603246|Female|27.0|        57000.0|        0|[27.0,57000.0]|
|15804002|  Male|19.0|        76000.0|        0|[19.0,76000.0]|
+--------+------+----+---------------+---------+--------------+
only showing top 5 rows



In [29]:
training_data, test_data = transformed_data.randomSplit([0.75, 0.25], seed = 0)

In [30]:
training_data.count()

301

In [31]:
from pyspark.ml.classification import LogisticRegression

In [32]:
lr = rfc = LogisticRegression(labelCol='Purchased', featuresCol='features')

In [33]:
lr_model  = lr.fit(training_data)

In [34]:
predictions = lr_model.transform(test_data)

In [35]:
predictions.show(5)

+--------+------+----+---------------+---------+---------------+--------------------+--------------------+----------+
| User ID|Gender| Age|EstimatedSalary|Purchased|       features|       rawPrediction|         probability|prediction|
+--------+------+----+---------------+---------+---------------+--------------------+--------------------+----------+
|15566689|Female|35.0|        57000.0|        0| [35.0,57000.0]|[2.01726864503134...|[0.88259828570303...|       0.0|
|15581198|  Male|31.0|        74000.0|        0| [31.0,74000.0]|[2.30539990032011...|[0.90932327007564...|       0.0|
|15581282|  Male|37.0|        74000.0|        0| [37.0,74000.0]|[1.04521839252925...|[0.73985564272230...|       0.0|
|15584320|Female|37.0|       146000.0|        1|[37.0,146000.0]|[-1.2926205482454...|[0.21540958613233...|       1.0|
|15584545|Female|32.0|        86000.0|        0| [32.0,86000.0]|[1.70572982555918...|[0.84628160233199...|       0.0|
+--------+------+----+---------------+---------+--------

In [36]:
pred = predictions.select('prediction').toPandas()

In [37]:
actual = predictions.select('Purchased').toPandas()

In [40]:
from sklearn.metrics import accuracy_score

In [41]:
accuracy_score(actual, pred)*100

86.86868686868688